# ch4 函數與模型

使用節點可以組織出複雜的運算系統，但節點也只是起了連接網路內各個元件的功能而已。支撐起整套運算系統的核心是模型和函數，下面我們就來簡單介紹一下兩者的差別:

**`函數:`**
1. 定義在unit_function模組中，功能是提供一些常見的函數，比如:tanh或是cross_entropy。
2. 使用時不需要事先導入到網路，直接在建立連接時使用即可。
3. 函數沒有需要優化的參數所以屬於資料節點。

**`模型:`**
1. 定義在layer模組中，功能是提供一些常見的(單層)類神經網路模型，比如:CNN或LSTM
2. 使用時需要導入到網路，否則運行節點時會保錯。
3. 模型需要優化的參數所以屬於模型節點。

In [10]:
from nyto import unit_function as uf
from nyto import layer
from nyto import net_tool as to

## 函數

函數有嚴格的使用規定，主要分成支援右移運算和不支援右移運算兩種，支援右移運算的可以寫成下面兩種形式:

    [1] node1 >> function(param,) >> node2
    [2] node2 = function(param,)(node1)

不支援右移運算只能寫成下面這種形式:
    
    [1] function(node1, param,) >> node2
    [2] node2 = function(node1, param,)
    
通常不支援右移運算的函數都是需要輸入兩個或兩個節點以上的函數，所以無法支援右移運算。

**`支援右移運算:`**
* variable_to_np()
* linear()
* relu()
* gaussian()
* sigmoid()
* tanh()
* col_nor()
* row_nor()
* softmax()
* global_average_pooling()
* global_max_pooling()
* flattening()
* max_pooling(kernel_shape_node_if, strides=1)
* average_pooling(kernel_shape_node_if, strides=1)

**`不支援右移運算:`**
* to_np(node_if)
* concatenate(*node_ifs, axis=1)
* tile(node_if, size_tuple)
* MSE(pre_if, target_if)
* RMSE(pre_if, target_if)
* MAE(pre_if, target_if)
* MAPE(pre_if, target_if)
* cross_entropy(pre_if, target_if)
* binary_cross_entropy(pre_if, target_if)
* accuracy(pre_if, target_if)

由於大部份的函數使用上都算直覺，這邊就挑幾個說明。

### max_pooling和average_pooling

這兩個函數是卷集神經網路會用到的pooling函數，需要調整的參數為:
* (tuple/node_interface)kernel_shape_node_if: 窗口的尺寸，比如(3,3)，也可以輸入一個節點界面。
* (int/node_interface)strides=1: 窗口移動的大小

In [11]:
import numpy as np

nn,n = to.new_net(
    # 輸入的圖片是4維的numpy.array
    # [第幾筆資料][channel][圖片row][圖片col]
    img_np = to.add_data(np.arange(2*3*5*5).reshape(2,3,5,5)),
    kernal_shape=(3,3),
    strides=1
)

n.img_np >> uf.max_pooling(n.kernal_shape, n.strides) >> n.max_pooling
n.img_np >> uf.average_pooling((3,3), strides=1) >> n.average_pooling

### concatenate和tile

這兩個函數是用來拼接numpy.array用的可以對應到numpy的函數:
* concatenate(*node_ifs, axis=1) -> np.concatenate(node_ifs, axis=axis)
* tile(node_if, size_tuple) -> np.tile(node_if, size_tuple)

In [12]:
nn,n = to.new_net(
    data1 = np.arange(9).reshape(3,3),
    data2 = np.arange(9,18).reshape(3,3),
    size_tuple = (3,2)
)

n.data3 = uf.concatenate(n.data1, n.data2)
n.data4 = uf.tile(n.data1, size_tuple=n.size_tuple)

In [13]:
to.get(n.data3)

array([[ 0,  1,  2,  9, 10, 11],
       [ 3,  4,  5, 12, 13, 14],
       [ 6,  7,  8, 15, 16, 17]])

In [14]:
to.get(n.data4)

array([[0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8],
       [0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8],
       [0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8]])

## 模型

下面來介紹模型，這邊需要注意的是前面我們介紹過模型節點，所謂的模型節點就是裡面保存了模型的節點。而模型節點最大的特點就是模型是可以被優化的，不能被優化的我們一般會被放到資料節點中。

這邊我們要做更近一步的說明了，首先:可以被優化的單元一定是模型，但模型不一定可以被優化。也就是說模型中其實也存在可以優化的模型與不能優化的模型。你可能會感到困惑，那資料與模型的差別在於什麼呢？

模型與資料的真正定義是:
> 可以被多個網路共享的單元為資料，各網路獨有的單元為模型

這說明模型與資料的差別並不是簡單的能不能優化而已，而在模型中存在著可以被優化與不能被優化的模型，我們將可以優化的模型稱為層模型(layer)。層模型被定義在`layer`模組中，有以下幾個成員:
1. variable_layer: 單純的矩陣，一般用於尋找最佳的輸入
2. nn_layer:       單層的類神經網路
3. lstm_layer:     單層的LSTM
4. conv_layer:     單層卷集層，裡面有多個keranl

### variable_layer

作為一個可優化的矩陣，能做到的事情遠比看上去的多。基本上，另外3種的層模型都可以使用該層去做出來，只不過會犧牲執行的效率。可以使用*layer.new_variable_layer*函數產生，下面是各項參數的說明:

`layer.new_variable_layer`

**structure**
    
    生成矩陣的形狀，資料型別是tuple。
    exp:(3,2)
**init_values**
    
    亂數生成時常態分配的平均數，預設0。
**random_size**
    
    亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    預設None。
**dropout**

    使用dropout訓練時是否啟用dropout，預設False。

如果你手邊有一個已經準備好的`numpy.array`，你可以使用*layer.np_to_variable_layer*轉換成`variable_layer`，下面是該函數的參數:

`layer.np_to_variable_layer`

**variable_np**
    
    要被轉換的numpy.array。    
**dropout**

    使用dropout訓練時是否啟用dropout，預設False。

需要注意的是`variable_layer`在網路中被執行時，如果需要與其他的不是層模型的模型互動時，建議先轉換成np.array。可以使用`unit_function.variable_to_np`或是`unit_function.to_np`做轉換:

In [15]:
nn,n = to.new_net(
    var1_layer=layer.new_variable_layer((3,3), 1), 
    var2_layer=layer.new_variable_layer((3,3), 2),
    nn_layer=layer.new_nn_layer((3,3)),
    data_np=np.arange(9).reshape(3,3)
)

In [16]:
# 與其他層模型互動時，不需要傳換
n.interact_with_other_layer1 = n.var1_layer + n.var2_layer
n.interact_with_other_layer2 = n.var1_layer >> n.nn_layer

to.get(n.interact_with_other_layer1, n.interact_with_other_layer2)

(variable_layer([[3. 3. 3.]
  [3. 3. 3.]
  [3. 3. 3.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

In [17]:
# 轉換方式1
n.use_conversion_1 = uf.to_np(n.var1_layer) + n.data_np

# 轉換方式2
n.use_conversion_2 = n.var1_layer >> uf.variable_to_np() >> n.nn_layer

to.get(n.use_conversion_1, n.use_conversion_2)

(array([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

### nn_layer

單層的神經網路，可以用來組合出複雜的網路結構，可以使用*layer.new_nn_layer*產生:

`layer.new_nn_layer`

**structure**

    網路結構，資料型別是tuple。
    第一格是網路的輸入大小，第一格是網路的輸出大小。
    exp:(3,2)  
**init_values**

    亂數生成時常態分配的平均數，資料型別是tuple，預設為(0,0)。
    第一格是網路權重的平均數，第二格是網路偏置的平均數。
**random_size**

    亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    第一格是網路權重的標準差，第二格是網路偏置的標準差。
    預設為(None, None)。
**dropout**
    
    使用dropout訓練時是否啟用dropout，預設True。

### lstm_layer

單層的神經網路，可以用來對時間序列分析，可以使用*layer.new_lstm_layer*產生:

`layer.new_random_lstm`

**structure**

    網路結構，資料型別是tuple。
    第一格是網路的輸入大小，第二格是網路的輸出大小。
    exp:(3,2)  
**init_values**

    亂數生成時常態分配的平均數，資料型別是tuple，預設為(0,0)。
    第一格是網路權重的平均數，第二格是網路偏置的平均數。
**random_size**

    亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    第一格是網路權重的標準差，第二格是網路偏置的標準差。
    預設為(None, None)。
**dropout**
    
    使用dropout訓練時是否啟用dropout，預設True。

### conv_layer

單層的卷集層，可以用來對圖像資料分析，可以使用*layer.new_conv_layer*產生:

`layer.new_random_lstm`

**structure**

    kernel結構，資料型別是tuple。
    第一格是kernel數量，第二格是kernel的row大小,第三格是kernel的col大小。
    exp:(2,3,3)
    
**init_values**

    亂數生成時常態分配的平均數，預設0。
**random_size**

    亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    預設None。
**dropout**
    
    使用dropout訓練時是否啟用dropout，資料型別是bool。
    預設False。
    
**pad_mod**

    padding模式，有三種選擇:(1)'full'(2)'valid'(3)'same'
    'same'=不做padding，預設為'valid'
    
**strides**

    窗口移動大小，資料型別是int。
    預設為1。

## 子網路

如果你手邊正好有一個訓練好的網路，它能不能與其他網路組合起來變成一個更好的網路呢？被其他網路導入的網路我們稱之為子網路，我們可以使用`net.push_get`來在其他網路中完成對該網路的使用:

In [18]:
# 子網路
sub_nn, sub_nn_node = to.new_net()
sub_nn_node.y = sub_nn_node.x1 + sub_nn_node.x2

# 導入子網路
nn, node = to.new_net(a=1, b=2, sub_nn=sub_nn)
node.sub_nn_return = node.sub_nn.push_get(
    {'y'}, x1=node.a, x2=node.b
)

to.get(node.sub_nn_return)

{'y': 3}

對於子網路，我們應該要如何看待呢？可以簡單當成層模型來使用。當優化時，被導入的子網路也會如同網路內的其他層模型一樣被優化。但如果只想使用而不想優化子網路，最簡單的方法就是在導入時使用資料的方式導入。當然更簡單的方式也是有的，使用者可以很方便的切換是否需要優化子網路，我們在下一章中再做介紹。

***

*END*